In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import json
import time
import csv
import scipy.io as sio
import collections
import gzip
from matplotlib import pyplot as plt

In [2]:
dir = '2020Sep24 - Overwatch League - 37 files/Event Stream Data/2020 Season Match Data/20200817/'
filename = f'{dir}payload_gameresult-20200810-20200816.tsv.gz'

In [3]:
df = pd.read_csv(filename, delimiter='\t')
new_df = pd.json_normalize(json.loads(df.loc[0,'players']))
len(df)
# new_df

46

In [14]:
def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

non_json_headers = ['time', 'winningteamid', 'total_game_time_ms', 'esports_match_id']
info_nested_json = ['esports_ids_esports_match_game_number', 'game_context_map_guid', 'game_context_map_type', 'game_context_round']
score_info_nested_json = ['attacking_team_esports_team_id']
team_info_nested_json = ['payload_distance', 'time_banked', 'score', 'control_info', 'team.esports_team_id']
players_info_nested_json = ['base_player.battletag', 'team.esports_team_id']

total_headers = non_json_headers + info_headers + info_nested_json + score_info_headers + score_info_nested_json + team_info_nested_json*2 + players_info_nested_json

n = 0
with gzip.open(filename, mode='rt', newline='') as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter = '\t')
    for row in tqdm(csv_reader, desc = 'num row'):
        n += 1
        esports_match_id = row['esports_match_id'] # esports_match_id 따로 빼서 비교
        # table_name = f'match_{esports_match_id}'
        data = [row[x] for x in non_json_headers]
        
        info_json = json.loads(row['info'])
        tmp = flatten_json(info_json)
        flat = pd.json_normalize(tmp)
        data.extend([(flat[x][0]) for x in info_nested_json])

        score_info_json = json.loads(row['score_info'])
        tmp = flatten_json(score_info_json)
        flat = pd.json_normalize(tmp)
        data.extend([(flat[x][0]) for x in score_info_nested_json])

        team_a = pd.json_normalize(tmp_flatten['team_info'][0][0])
        team_b = pd.json_normalize(tmp_flatten['team_info'][0][1])
        data.extend([team_a[x][0] for x in team_info_nested_json])
        data.extend([team_b[x][0] for x in team_info_nested_json])

        players_json = json.loads(row['players'])
        tmp_flatten = pd.json_normalize(players_json)
        data.extend([(tmp_flatten[x]) for x in players_info_nested_json])

num row: 46it [00:00, 167.73it/s]


In [16]:
def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out
sample_json = score_info_json
flat = flatten_json(sample_json)
res = pd.json_normalize(flat)
res

,round_num,attacking_team_id,round_name_guid,team_info_0_team_id,team_info_0_payload_distance,team_info_0_time_banked,team_info_0_score,team_info_0_control_info,team_info_0_team_team_id,team_info_0_team_esports_team_id,team_info_1_team_id,team_info_1_payload_distance,team_info_1_time_banked,team_info_1_score,team_info_1_control_info,team_info_1_team_team_id,team_info_1_team_esports_team_id,attacking_team_team_id,attacking_team_esports_team_id
0,4,3dfb307c-5c7a-bc48-9e41-96a3cbcc414f,0,3dfb307c-5c7a-bc48-9e41-96a3cbcc414f,0.0,0.0,3,None,3dfb307c-5c7a-bc48-9e41-96a3cbcc414f,7698,6df3c959-372b-954b-a03b-211d9d80c089,0.0,0.0,4,None,6df3c959-372b-954b-a03b-211d9d80c089,7696,3dfb307c-5c7a-bc48-9e41-96a3cbcc414f,7698


In [9]:
score_info_json

{'round_num': 4,
 'attacking_team_id': '3dfb307c-5c7a-bc48-9e41-96a3cbcc414f',
 'round_name_guid': 0,
 'team_info': [{'team_id': '3dfb307c-5c7a-bc48-9e41-96a3cbcc414f',
   'payload_distance': 0.0,
   'time_banked': 0.0,
   'score': 3,
   'control_info': None,
   'team': {'team_id': '3dfb307c-5c7a-bc48-9e41-96a3cbcc414f',
    'esports_team_id': 7698}},
  {'team_id': '6df3c959-372b-954b-a03b-211d9d80c089',
   'payload_distance': 0.0,
   'time_banked': 0.0,
   'score': 4,
   'control_info': None,
   'team': {'team_id': '6df3c959-372b-954b-a03b-211d9d80c089',
    'esports_team_id': 7696}}],
 'attacking_team': {'team_id': '3dfb307c-5c7a-bc48-9e41-96a3cbcc414f',
  'esports_team_id': 7698}}

In [15]:
data

['1597622085567',
 '6df3c959-372b-954b-a03b-211d9d80c089',
 '1322694',
 '34765',
 3,
 576460752303423963,
 'ASSAULT',
 4,
 7698,
 0.0,
 0.0,
 3,
 None,
 7698,
 0.0,
 0.0,
 4,
 None,
 7696,
 0         Saucy#7530
 1        Dalton#6004
 2          KSAA#3999
 3         Roolf#5920
 4         Gator#1190
 5          Hawk#3240
 6     ShRedLock#9774
 7         Masaa#5436
 8        Edison#8117
 9          lr1s#6266
 10       CarCar#2891
 11    sHockWave#4050
 Name: base_player.battletag, dtype: object,
 0     7698
 1     7696
 2     7696
 3     7696
 4     7698
 5     7698
 6     7696
 7     7698
 8     7698
 9     7698
 10    7696
 11    7696
 Name: team.esports_team_id, dtype: int64]